# Uptrend Prompts
- The following is SPY stock 30m data with Open/High/Low/Close. I want to create some possible data pattern based on assumptions, you need to be faily random, I prefer you use LLM to predict rather than apply fixed algorithm; IF  you used program, show me the program.  please give timestampes with the same increments as input. The data are US stock trading hours, so  generate for the next 30 days during those hours
  - up by no more than 10% from the last data, may reache that early and then drop 
  - daily swing is no more than 2.5%
  - generate each row of data with 30 mins incremental


In [ ]:
import random

# Starting parameters
last_close = 567
max_high = last_close * 1.10  # 623.7
days = 30
intervals_per_day = 13
timestamp_increment = 1800  # 30 minutes
day_jump = 64800  # between trading days

# Generate one day's data
def generate_day(day_num, prev_close, start_timestamp):
    open_price = prev_close * (1 + random.uniform(-0.005, 0.005))
    swing_limit = open_price * 0.025
    if day_num <= 5:
        target_close = prev_close * (1 + random.uniform(0.005, 0.02))
        daily_high = min(max_high, prev_close + random.uniform(5, 15))
    else:
        target_close = prev_close * (1 - random.uniform(0.005, 0.015))
        daily_high = prev_close + random.uniform(0, 5)
    daily_low = max(daily_high - swing_limit, open_price - swing_limit)
    
    intervals = []
    current_price = open_time = open_price
    timestamp = start_timestamp
    for i in range(intervals_per_day):
        if i < 3 and day_num <= 5:
            high = min(daily_high, current_price + random.uniform(0, 5))
        else:
            high = current_price + random.uniform(-1, 2)
        low = max(daily_low, current_price - random.uniform(0, 3))
        close = current_price + (target_close - current_price) * (i + 1) / intervals_per_day
        if i == intervals_per_day - 1:
            close = target_close
        intervals.append(f"{timestamp},{open_time:.2f},{high:.2f},{low:.2f},{close:.2f}")
        open_time = current_price = close
        timestamp += timestamp_increment
    return intervals

# Generate 30 days
data = []
current_timestamp = 1742239800 + day_jump
for day in range(1, 31):
    day_data = generate_day(day, last_close, current_timestamp)
    data.extend(day_data)
    last_close = float(day_data[-1].split(',')[-1])
    current_timestamp += day_jump

# Output sample
for row in data[:26]:  # First 2 days
    print(row)

In [ ]:
pip install mplfinance

In [ ]:
import random
import math
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime

# Starting parameters
last_close = 567
max_high = last_close * 1.10  # 623.7
days = 30
intervals_per_day = 13
timestamp_increment = 1800  # 30 minutes
day_jump = 64800  # between trading days

# Generate one day's data
def generate_day(day_num, prev_close, start_timestamp):
    open_price = prev_close * (1 + random.uniform(-0.005, 0.005))
    swing_limit = open_price * 0.025
    if day_num <= 5:
        target_close = prev_close * (1 + random.uniform(0.005, 0.02))
        daily_high = min(max_high, prev_close + random.uniform(5, 15))
    else:
        target_close = prev_close * (1 - random.uniform(0.005, 0.015))
        daily_high = prev_close + random.uniform(0, 5)
    daily_low = max(daily_high - swing_limit, open_price - swing_limit)
    
    # Define amplitude for the wave
    amplitude = 5  # some value to determine the wave magnitude
    intervals = []
    current_price = open_time = open_price
    timestamp = start_timestamp
    for i in range(intervals_per_day):
        # Calculate linear trend component
        trend_component = (target_close - open_price) * (i + 1) / intervals_per_day
        # Calculate wave component that starts and ends at zero
        wave_i = amplitude * math.sin(math.pi * i / (intervals_per_day - 1))
        # Calculate close with wave component
        close = open_price + trend_component + wave_i
        # Ensure close is within daily limits
        close = max(daily_low, min(daily_high, close))
        # Set high and low based on open and close
        if close > open_time:
            high = close + random.uniform(0, 1)
            low = open_time - random.uniform(0, 1)
        else:
            high = open_time + random.uniform(0, 1)
            low = close - random.uniform(0, 1)
        # Ensure high and low are within daily limits
        high = min(daily_high, high)
        low = max(daily_low, low)
        # If it's one of the first 3 intervals and day_num <= 5, adjust high to be higher
        if i < 3 and day_num <= 5:
            high = min(daily_high, high + random.uniform(0, 2))
        intervals.append((timestamp, open_time, high, low, close))
        open_time = current_price = close
        timestamp += timestamp_increment
    return intervals

# Generate 30 days
data = []
current_timestamp = 1742239800 + day_jump
for day in range(1, 31):
    day_data = generate_day(day, last_close, current_timestamp)
    data.extend(day_data)
    last_close = day_data[-1][-1]
    current_timestamp += day_jump

# Output sample for first 2 days
for row in data[:26]:
    print(f"{row[0]},{row[1]:.2f},{row[2]:.2f},{row[3]:.2f},{row[4]:.2f}")

# Plot the closing prices
timestamps = [row[0] for row in data]
closing_prices = [row[4] for row in data]

# Convert timestamps to datetime objects
dates = [datetime.fromtimestamp(ts) for ts in timestamps]

plt.figure(figsize=(12, 6))
plt.plot(dates, closing_prices, label='Closing Price')
plt.xlabel('Date')
plt.ylabel('Price')
plt.title('Generated Stock Prices Over 30 Days')
plt.legend()

# Format the x-axis to show dates
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=2))
plt.gcf().autofmt_xdate()

plt.show()

In [ ]:
import pandas as pd

In [ ]:
import numpy as np
from datetime import datetime, timedelta

# Configuration
last_close = 567.0
max_allowed = last_close * 1.10  # 623.7
num_days = 30

# Generate timestamps (UTC for NYSE trading hours)
def generate_timestamps(start_time, num_days):
    current = datetime.utcfromtimestamp(start_time)
    timestamps = []
    days_added = 0
    while days_added < num_days:
        current += timedelta(days=1)
        if current.weekday() < 5:  # Skip weekends
            for i in range(13):  # 13 intervals/day (9:30 AM - 4:00 PM ET)
                interval = current.replace(hour=13, minute=30) + timedelta(minutes=30*i)
                timestamps.append(int(interval.timestamp()))
            days_added += 1
    return timestamps

# Price generator with daily bounds
def generate_ohlc(prev_close, daily_high, daily_low, num_intervals=13):
    opens = [prev_close]
    closes = []
    for _ in range(num_intervals-1):
        change = np.random.uniform(-0.002, 0.002)  # ±0.2% per 30m
        new_close = opens[-1] * (1 + change)
        new_close = np.clip(new_close, daily_low, daily_high)
        closes.append(new_close)
        opens.append(new_close)
    highs = [max(o, c) + np.random.uniform(0, 0.001)*o for o, c in zip(opens, closes)]
    lows = [min(o, c) - np.random.uniform(0, 0.001)*o for o, c in zip(opens, closes)]
    return list(zip(opens, highs, lows, closes))

# Generate data
timestamps = generate_timestamps(1742239800, num_days)
data = []
current_close = last_close
for ts in timestamps:
    if timestamps.index(ts) % 13 == 0:  # New day
        daily_high = min(current_close * 1.025, max_allowed)
        daily_low = max(current_close * 0.975, daily_high * 0.975)
    ohlc = generate_ohlc(current_close, daily_high, daily_low)
    for entry in ohlc:
        data.append([ts, entry[0], entry[1], entry[2], entry[3]])
        ts += 1800  # Add 30 mins
    current_close = ohlc[-1][3]

# Convert data to DataFrame for plotting
df = pd.DataFrame(data, columns=['time', 'open', 'high', 'low', 'close'])
df['time'] = pd.to_datetime(df['time'], unit='s')

# Plot the OHLC data
mpf.plot(df.set_index('time'), type='ohlc', style='candlestick', title='Generated Stock Data', ylabel='Price')

# Save to CSV
import csv
import pandas as pd
import mplfinance as mpf
with open('generated_spy_30m.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['time', 'open', 'high', 'low', 'close'])
    writer.writerows(data)